# Microsoft DialoGPT

In [1]:
#importing libraries
import numpy as np
import time
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
# checkpoint
checkpoint = "microsoft/DialoGPT-medium"
# download and cache tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# download and cache pre-trained model
model = AutoModelForCausalLM.from_pretrained(checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
# Build a ChatBot class with all necessary modules to make a complete conversation
class ChatBot():
    # initialize
    def __init__(self):
        # once chat starts, the history will be stored for chat continuity
        self.chat_history_ids = None
        # make input ids global to use them anywhere within the object
        self.bot_input_ids = None
        # a flag to check whether to end the conversation
        self.end_chat = False
        # greet while starting
        self.welcome()

    def welcome(self):
        print("Initializing ChatBot ...")
        print('Type "bye" or "quit" or "exit" to end chat \n')
        # Greet and introduce
        greeting = np.random.choice([
            "Welcome, I am ChatBot, here for your kind service",
            "Hey, Great day! I am your virtual assistant",
            "Hello, it's my pleasure meeting you",
            "Hi, I am a ChatBot. Let's chat!"
        ])
        print("ChatBot >>  " + greeting)

    def user_input(self):
        # receive input from user
        text = input("User    >> ")
        # end conversation if user wishes so
        if text.lower().strip() in ['bye', 'quit', 'exit']:
            self.end_chat=True
            print('ChatBot >>  See you soon! Bye!')
        else:
            # continue chat, preprocess input text
            # encode the new user input, add the eos_token and return a tensor in Pytorch
            self.new_user_input_ids = tokenizer.encode(text + tokenizer.eos_token, \
                                                       return_tensors='pt')

    def bot_response(self):
        # append the new user input tokens to the chat history
        # if chat has already begun
        if self.chat_history_ids is not None:
            self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1)
        else:
            # if first entry, initialize bot_input_ids
            self.bot_input_ids = self.new_user_input_ids

        # define the new chat_history_ids based on the preceding chats
        # generated a response while limiting the total chat history to 1000 tokens,
        self.chat_history_ids = model.generate(self.bot_input_ids, max_length=1000, \
                                               pad_token_id=tokenizer.eos_token_id)

        # last ouput tokens from bot
        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], \
                               skip_special_tokens=True)
        # in case, bot fails to answer
        if response == "":
            response = self.random_response()
        print('ChatBot >>  '+ response)

    # in case there is no response from model
    def random_response(self):
        i = -1
        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # iterate over history backwards to find the last token
        while response == '':
            i = i-1
            response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # if it is a question
        if response.strip() == '?':
            reply = np.random.choice(["I don't know",
                                     "I am not sure"])
        # not a question?
        else:
            reply = np.random.choice(["Great",
                                      "Fine. What's up?",
                                      "Okay"
                                     ])
        return reply

In [4]:
# build a ChatBot object
bot = ChatBot()

while True:
    bot.user_input()
    if bot.end_chat:
        break
    bot.bot_response()

Initializing ChatBot ...
Type "bye" or "quit" or "exit" to end chat 

ChatBot >>  Hi, I am a ChatBot. Let's chat!
User    >> hi, how are you


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


ChatBot >>  I'm good, how are you?
User    >> what languages do you knowi
ChatBot >>  I know a little bit of English, but I'm not fluent in it.
User    >> can you say good morning in korean
ChatBot >>  I can, but I don't know how to speak it.
User    >> can you write then
ChatBot >>  I can't, but I can speak it.
User    >> translate good morning to korean
ChatBot >>  I can't, but I can speak it.
User    >> tell me a joke
ChatBot >>  I can't, but I can speak it.
User    >> bye
ChatBot >>  See you soon! Bye!


# OpenAI

In [ ]:
!pip install openai==0.28.0

In [ ]:
#import libraries
import openai
import os

In [ ]:
# Set API key here
openai.api_key = ""

In [ ]:
class OpenAIChatBot:
    def __init__(self):
        self.chat_history = []  # Store conversation history for context
        self.end_chat = False
        self.welcome()

    def welcome(self):
        print("Initializing OpenAI ChatBot ...")
        print('Type "bye", "quit", or "exit" to end the chat.\n')
        print("ChatBot >> Hello! I'm here to assist you. What can I help you with today?")

    def user_input(self):
        # Get user input
        text = input("User    >> ")
        if text.lower().strip() in ['bye', 'quit', 'exit']:
            self.end_chat = True
            print("ChatBot >> Goodbye! Have a great day!")
        return text

    def get_response(self, user_input):
        # Add user input to chat history
        self.chat_history.append({"role": "user", "content": user_input})

        # Prepare the messages in the required format
        messages = [{"role": "system", "content": "Hi."}] + self.chat_history

        try:
            # Use the new API call
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=messages,
                max_tokens=150,  # Limit response length
                temperature=0.8  # Adjust creativity
            )

            # Extract the assistant's reply
            assistant_reply = response["choices"][0]["message"]["content"]

            # Add the assistant's response to chat history
            self.chat_history.append({"role": "assistant", "content": assistant_reply})
            return assistant_reply
        except Exception as e:
            return f"Error: {e}"

    def chat(self):
        while not self.end_chat:
            user_input = self.user_input()
            if not self.end_chat:
                response = self.get_response(user_input)
                print(f"ChatBot >> {response}")


In [ ]:

# Create an instance of the chatbot
chatbot = OpenAIChatBot()

# Start the chat
chatbot.chat()

Initializing OpenAI ChatBot ...
Type "bye", "quit", or "exit" to end the chat.

ChatBot >> Hello! I'm here to assist you. What can I help you with today?
User    >> hi, how are you
ChatBot >> Hello! As an artificial intelligence, I don't have feelings, but I'm here and ready to assist you. How can I help you today?
User    >> do you know any languages
ChatBot >> Yes, as an artificial intelligence, I have the ability to understand and generate text in multiple languages such as English, Spanish, French, German, Italian, Dutch, Russian, Chinese, Japanese, Korean, and many more. How can I assist you today?
User    >> how do you good morning in italian
ChatBot >> In Italian, "Good morning" is "Buongiorno".
User    >> what about how are you
ChatBot >> In Italian, "How are you?" is "Come stai?"
User    >> can you say a joke
ChatBot >> Sure, here's a classic for you:

Why don't scientists trust atoms?

Because they make up everything!
User    >> can you write a code print pattern of triangle 